### **This Notebook Performs a Parameters Search on Every Possible Cleaning and Model Parameter.**

Lets test different spectral cleaning parameters by building a model pipeline.

Import Libraries

In [1]:
import pandas as pd
import numpy as np
from scipy.signal import savgol_filter
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import cross_validate, GroupKFold, cross_val_score
from Spectra_Preparation_Functions import *
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
import optuna
from joblib import Parallel, delayed
import csv

Read the spectral data

In [2]:
#df = pd.read_csv("../../data/exosomes.raw_spectrum_1.csv")
df = pd.read_csv("../../data/exosomes.raw_spectrum_400-1800.csv")
#df = pd.read_csv("../../data/400-1800_spike_removed.csv")

In [3]:
df

,SpecID,Seq,WaveNumber,Absorbance,SurID,Status
0,201210-1-00,293,400.22778,1765.6628,201210-1,Normal
1,201210-1-00,294,400.91116,1774.7809,201210-1,Normal
2,201210-1-00,295,401.59454,1769.0302,201210-1,Normal
3,201210-1-00,296,402.27789,1756.4220,201210-1,Normal
4,201210-1-00,297,402.96127,1758.8690,201210-1,Normal
...,...,...,...,...,...,...
6239200,210526-3-09,2337,1797.03870,1617.3926,210526-3,Hyperglycemia
6239201,210526-3-09,2338,1797.72200,1633.0911,210526-3,Hyperglycemia
6239202,210526-3-09,2339,1798.40550,1633.3076,210526-3,Hyperglycemia
6239203,210526-3-09,2340,1799.08890,1641.8665,210526-3,Hyperglycemia


In [4]:
wavelength_df = prepare_wavelength_df(df, 'Absorbance')
wavelength_df.head()

,400.22778,400.91116,401.59454,402.27789,402.96127,403.64465,404.32803,405.01138,405.69476,406.37814,...,1794.9886,1795.672,1796.3553,1797.0387,1797.722,1798.4055,1799.0889,1799.7722,SurID,Status
SpecID,,,,,,,,,,,,,,,,,,,,,
201210-1-00,1765.6628,1774.7809,1769.0302,1756.4220,1758.8690,1763.2300,1745.2285,1773.3534,1774.7166,1753.3281,...,1210.4993,1213.9619,1225.2153,1210.0010,1210.6858,1194.4679,1195.1451,1189.8683,201210-1,Normal
201210-1-01,1966.9930,1962.4237,1954.5616,1954.3228,1963.0917,1975.0807,1979.3162,1963.4561,1968.4587,1964.0000,...,1382.6973,1363.7004,1360.6210,1354.0477,1353.0381,1353.9978,1361.2426,1370.2874,201210-1,Normal
201210-1-02,2182.6694,2149.6565,2146.0227,2159.3459,2167.2910,2160.9861,2145.6575,2134.2004,2142.8303,2138.6309,...,1976.2070,1989.0183,1996.2838,1979.3507,1976.2002,1994.9839,1974.2030,1971.1880,201210-1,Normal
201210-1-03,2445.0837,2430.4973,2422.7927,2434.3433,2454.9700,2462.8245,2454.7007,2467.7329,2449.5161,2421.3474,...,1992.3817,2022.6331,2001.8311,2010.0946,2006.4933,2017.2891,2038.1699,2000.6475,201210-1,Normal
201210-1-04,2250.4536,2248.6235,2245.0984,2242.7173,2235.2803,2228.9585,2236.0095,2229.6091,2225.9231,2211.0359,...,2009.0385,1953.3303,1963.5698,1964.5299,1969.5634,1986.6266,1970.1484,2007.0848,201210-1,Normal


>#### **Use Optuna to Find the Best Cleaning Parameter**

In [5]:
df

,SpecID,Seq,WaveNumber,Absorbance,SurID,Status
0,201210-1-00,293,400.22778,1765.6628,201210-1,Normal
1,201210-1-00,294,400.91116,1774.7809,201210-1,Normal
2,201210-1-00,295,401.59454,1769.0302,201210-1,Normal
3,201210-1-00,296,402.27789,1756.4220,201210-1,Normal
4,201210-1-00,297,402.96127,1758.8690,201210-1,Normal
...,...,...,...,...,...,...
6239200,210526-3-09,2337,1797.03870,1617.3926,210526-3,Hyperglycemia
6239201,210526-3-09,2338,1797.72200,1633.0911,210526-3,Hyperglycemia
6239202,210526-3-09,2339,1798.40550,1633.3076,210526-3,Hyperglycemia
6239203,210526-3-09,2340,1799.08890,1641.8665,210526-3,Hyperglycemia


In [6]:
def objective(trial):

    # Values for each hyperparameter
    despike = trial.suggest_categorical('despike', [True, False])
    despike_ma = trial.suggest_int('despike_ma', 5, 100)
    despike_threshold = trial.suggest_float('despike_threshold', 3, 10, step=0.25)
    lam = trial.suggest_categorical('lam', [10**2, 10**3, 10**4, 10**5, 10**6, 10**7, 10**8, 10**9])
    p = trial.suggest_float('p', 0.001, 0.1, step=0.001)
    window_size = trial.suggest_int('window_size', 5, 201)
    poly_order = trial.suggest_int('poly_order', 1, 4)
    scaling = trial.suggest_categorical('scaling', [False, 'normal', 'vector', 'svn'])

    df_copy = df.copy()

    # Apply preprocessing based on suggested parameters
    if despike:
        df_copy['Absorbance'] = df_copy.groupby('SpecID')['Absorbance'].transform(lambda x: despike_group(x, ma=despike_ma, threshold = despike_threshold))
    df_copy['Baseline'] = df_copy.groupby('SpecID')['Absorbance'].transform(lambda x: asls_baseline_correction(x, lam=lam, p=p))
    df_copy['Baseline_Corrected_Absorbance'] = df_copy['Absorbance'] - df_copy['Baseline']
    df_copy['Smooth_Baseline_Corrected'] = df_copy.groupby('SpecID')['Baseline_Corrected_Absorbance'].transform(lambda x: savgol_filter(x, window_size, poly_order, deriv=0))
    if scaling:
        if scaling == 'normal':
            df_copy['Smooth_Baseline_Corrected'] = df_copy.groupby('SpecID')['Smooth_Baseline_Corrected'].transform(lambda x: normalise(x))
        elif scaling == 'vector':
            df_copy['Smooth_Baseline_Corrected'] = df_copy.groupby('SpecID')['Smooth_Baseline_Corrected'].transform(lambda x: vector_normalise(x))
        else:
            df_copy['Smooth_Baseline_Corrected'] = df_copy.groupby('SpecID')['Smooth_Baseline_Corrected'].transform(lambda x: svn_normalise(x))

    # Prepare data for ML
    wavelength_df = prepare_wavelength_df(df_copy, 'Smooth_Baseline_Corrected')
    X = wavelength_df.drop(['Status', 'SurID'], axis=1)
    y = wavelength_df['Status']
    groups = wavelength_df['SurID']

    # Classifier and cross-validation setup
    et = ExtraTreesClassifier(random_state=1234)
    cv = GroupKFold(n_splits=10)

    # Perform cross-validation
    scores = cross_validate(et, X, y, groups=groups, cv=cv, scoring='accuracy')
    #scores = cross_val_score(et, X, y, groups=groups, cv=cv, scoring='accuracy')

    # Return the average accuracy across all folds
    return np.mean(scores['test_score'])


# sampler = TPESampler(seed=10)  # Make the sampler behave in a deterministic way.
# study = optuna.create_study(direction='maximise', sampler=sampler)

study = optuna.create_study(direction="maximize")
# study.optimize(objective, n_trials=100, n_jobs=-1)
study.optimize(objective, n_trials=200, n_jobs=-1)

print(study.best_trial)

[I 2024-03-28 17:47:19,029] A new study created in memory with name: no-name-edf9bd69-2ad0-468c-bd22-ea7689db31ab
[I 2024-03-28 17:54:06,879] Trial 10 finished with value: 0.5870476190476189 and parameters: {'despike': False, 'despike_ma': 99, 'despike_threshold': 7.75, 'lam': 100000000, 'p': 0.055, 'window_size': 78, 'poly_order': 2, 'scaling': False}. Best is trial 10 with value: 0.5870476190476189.
[I 2024-03-28 17:54:13,338] Trial 0 finished with value: 0.5168730158730158 and parameters: {'despike': False, 'despike_ma': 41, 'despike_threshold': 4.25, 'lam': 100, 'p': 0.046, 'window_size': 46, 'poly_order': 2, 'scaling': False}. Best is trial 10 with value: 0.5870476190476189.
[I 2024-03-28 17:54:16,135] Trial 5 finished with value: 0.5731587301587301 and parameters: {'despike': False, 'despike_ma': 30, 'despike_threshold': 3.0, 'lam': 100000000, 'p': 0.053000000000000005, 'window_size': 45, 'poly_order': 3, 'scaling': 'vector'}. Best is trial 10 with value: 0.5870476190476189.
[I 2

FrozenTrial(number=156, state=1, values=[0.6113015873015873], datetime_start=datetime.datetime(2024, 3, 28, 19, 1, 13, 971042), datetime_complete=datetime.datetime(2024, 3, 28, 19, 12, 24, 403463), params={'despike': True, 'despike_ma': 34, 'despike_threshold': 9.5, 'lam': 1000000000, 'p': 0.035, 'window_size': 60, 'poly_order': 4, 'scaling': 'svn'}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'despike': CategoricalDistribution(choices=(True, False)), 'despike_ma': IntDistribution(high=100, log=False, low=5, step=1), 'despike_threshold': FloatDistribution(high=10.0, log=False, low=3.0, step=0.25), 'lam': CategoricalDistribution(choices=(100, 1000, 10000, 100000, 1000000, 10000000, 100000000, 1000000000)), 'p': FloatDistribution(high=0.1, log=False, low=0.001, step=0.001), 'window_size': IntDistribution(high=201, log=False, low=5, step=1), 'poly_order': IntDistribution(high=4, log=False, low=1, step=1), 'scaling': CategoricalDistribution(choices=(False, 'norma

In [25]:
results_df = study.trials_dataframe(attrs=("number", "value", "params", "state"))
results_df.to_csv("../../data/studies/et_optuna2-3.csv")

In [26]:
import joblib

joblib.dump(study, "../../data/studies/cleaning_study2.pkl")

# loaded_study = joblib.load("../../data/studies/cleaning_study.pkl")
# print("Best trial until now:")
# print(" Value: ", loaded_study.best_trial.value)
# print(" Params: ")
# for key, value in loaded_study.best_trial.params.items():
#     print(f"    {key}: {value}")

['../../data/studies/cleaning_study2.pkl']

In [20]:
print(study.best_trial)

FrozenTrial(number=231, state=1, values=[0.6161111111111112], datetime_start=datetime.datetime(2024, 3, 28, 20, 34, 8, 637343), datetime_complete=datetime.datetime(2024, 3, 28, 20, 44, 47, 204986), params={'despike': True, 'despike_ma': 78, 'despike_threshold': 5.75, 'lam': 1000000000, 'p': 0.041, 'window_size': 73, 'poly_order': 4, 'scaling': 'svn'}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'despike': CategoricalDistribution(choices=(True, False)), 'despike_ma': IntDistribution(high=100, log=False, low=5, step=1), 'despike_threshold': FloatDistribution(high=10.0, log=False, low=3.0, step=0.25), 'lam': CategoricalDistribution(choices=(100, 1000, 10000, 100000, 1000000, 10000000, 100000000, 1000000000)), 'p': FloatDistribution(high=0.1, log=False, low=0.001, step=0.001), 'window_size': IntDistribution(high=201, log=False, low=5, step=1), 'poly_order': IntDistribution(high=4, log=False, low=1, step=1), 'scaling': CategoricalDistribution(choices=(False, 'norm

In [21]:
from optuna.visualization import plot_optimization_history, plot_slice

plot_optimization_history(study)

In [22]:
plot_slice(study)

In [23]:
optuna.visualization.plot_contour(study)

In [24]:
optuna.visualization.plot_param_importances(study)

Continue the study.

In [19]:
study.optimize(objective, n_trials=200, n_jobs=-1)

print(study.best_trial)

[I 2024-03-29 00:44:53,997] Trial 307 finished with value: 0.6072698412698412 and parameters: {'despike': True, 'despike_ma': 22, 'despike_threshold': 5.0, 'lam': 1000000000, 'p': 0.043000000000000003, 'window_size': 75, 'poly_order': 4, 'scaling': False}. Best is trial 231 with value: 0.6161111111111112.
[I 2024-03-29 00:45:11,190] Trial 309 finished with value: 0.5673174603174603 and parameters: {'despike': True, 'despike_ma': 95, 'despike_threshold': 9.0, 'lam': 1000000000, 'p': 0.043000000000000003, 'window_size': 76, 'poly_order': 4, 'scaling': 'normal'}. Best is trial 231 with value: 0.6161111111111112.
[I 2024-03-29 00:45:11,836] Trial 308 finished with value: 0.5762698412698413 and parameters: {'despike': True, 'despike_ma': 69, 'despike_threshold': 9.0, 'lam': 1000000000, 'p': 0.043000000000000003, 'window_size': 67, 'poly_order': 4, 'scaling': 'normal'}. Best is trial 231 with value: 0.6161111111111112.
[I 2024-03-29 00:45:12,071] Trial 310 finished with value: 0.570269841269

KeyboardInterrupt: 